In [1]:
import numpy as np
import cv2

In [2]:
a = np.zeros((4, 2), dtype = 'float32')

In [3]:
a

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)

In [4]:
b = np.array([[1, 3], [3, 3], [1, 1], [3, 1]])

In [5]:
b

array([[1, 3],
       [3, 3],
       [1, 1],
       [3, 1]])

In [7]:
s = b.sum(axis = 1)

In [8]:
s

array([4, 6, 2, 4])

In [9]:
a[0] = b[np.argmin(s)]

In [10]:
a

array([[1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)

In [11]:
a[1] = b[np.argmax(s)]

In [12]:
a

array([[1., 1.],
       [3., 3.],
       [0., 0.],
       [0., 0.]], dtype=float32)

In [13]:
diff = np.diff(b, axis = 1)

In [14]:
diff

array([[ 2],
       [ 0],
       [ 0],
       [-2]])

In [ ]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype = 'float32')

    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    return rect

In [15]:
import numpy as np
import cv2

def order_points(pts):
    rect = np.zeros((4, 2), dtype = 'float32')

    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    return rect

def auto_scan_image_via_webcam():
    try:
        cap = cv2.VideoCapture(0)
    except:
        print('Cannot load camera!')
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print('Cannot load camera!')
            break

        k = cv2.waitKey(10)
        if k == 27:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (3, 3), 0)
        edged = cv2.Canny(gray, 75, 200)

        print("STEP1: Edge Detection")

        cnts, _ = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

        for c in cnts:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            screenCnt = []

            if len(approx) == 4:
                contourSize = cv2.contourArea(approx)
                camSize = frame.shape[0] * frame.shape[1]
                ratio = contourSize / camSize
                print(f"Contour size: {contourSize}")
                print(f"Camera size: {camSize}")
                print(f"Contour / Cam ratio: {ratio}")

                if ratio > 0.1:
                    screenCnt = approx

                break

        if len(screenCnt) == 0:
            cv2.imshow("WebCam", frame)
            continue

        else:
            print("STEP2: Find Contours")

            cv2.drawContours(frame, [screenCnt], -1, (0, 255, 0), 2)
            cv2.imshow("WebCame", frame)

            rect = order_points(screenCnt.reshape(4, 2))
            topLeft, topRight, bottomRight, bottomLeft = rect

            w1 = abs(bottomRight[0] - bottomLeft[0])
            w2 = abs(topRight[0] - topLeft[0])
            h1 = abs(topRight[1] - bottomRight[1])
            h2 = abs(topLeft[1] - bottomLeft[1])

            maxWidth = max([w1, w2])
            maxHeight = max([h1, h2])

            dst = np.float32([0, 0], [maxWidth-1, 0],
                             [maxWidth-1, maxHeight-1], [0, maxHeight-1])

            M = cv2.getPerspectiveTransform(rect, dst)
            warped = cv2.warpPerspective(frame, M, (maxWidth, maxHeight))

            print("STEP 3: Apply perspective transform")

            warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
            warped = cv2.adaptiveThreshold(warped, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10)

            print("STEP 4: Apply Adaptive Threshold")

            break

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

    cv2.imshow("Scanned", warped)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == "__main__":
    auto_scan_image_via_webcam()


Cannot load camera!


UnboundLocalError: local variable 'warped' referenced before assignment